In [1]:
import pandas as pd
import numpy as np
import os
import copy
import json
from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

In [2]:
data_folder = '../data_Megogo/'
new_data_folder = '../generated_data/'
model_folder = new_data_folder +'models/'
dic_folder = new_data_folder+'dic/'

In [3]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

# Train dataset

In [38]:
train = pd.read_csv(new_data_folder+'train_first_level.csv')
train[['user', 'item']] = train[['user', 'item']].astype(str)
train.columns = ['user_id', 'video_id', 'watching_percentage']
train.head(2)

,user_id,video_id,watching_percentage
0,21603820,9583642,0.0839
1,35636970,24645936,0.5990


In [39]:
train.head(5)

,user_id,video_id,watching_percentage
0,21603820,9583642,0.0839
1,35636970,24645936,0.5990
2,78312976,25397362,1.0000
3,122261599,5205267,0.9100
4,53477088,14098190,0.4850


In [5]:
train = train.drop(['watching_percentage'], axis=1)
train = train.drop_duplicates()

In [7]:
val_count = train.primary_video_id.value_counts()

In [9]:
val = list(val_count.values)

In [32]:
trace = go.Bar(
    x=list(range(len(val)))[:1500],
    y=val[:1500],
    name='Distribution',
#     marker=dict(
#         color='black'
#     )
)


data = [trace]
layout = go.Layout(
    title='Розподіл кількості переглядів фільмів',
    annotations=[
        dict(
            x=150,
            y=20000,
            xref='x',
            yref='y',
            text='20% of all films',
            showarrow=True,
            font=dict(
                family='Courier New, monospace',
                size=12,
                color='#ffffff'
            ),
            align='center',
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor='#636363',
            ax=20,
            ay=-30,
            bordercolor='#c7c7c7',
            borderwidth=2,
            borderpad=4,
            bgcolor='#ff7f0e',
            opacity=0.8
        )]
)
layout['shapes'] = [{
            'type': 'line',
            'x0': 150,
            'y0': 0,
            'x1': 150,
            'y1': 20000,
            'line': {
                'color': 'rgb(219, 10, 10)',
                'width': 3,
            },
        }]

config={'showLink': False, 'displayModeBar': False}

fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='plots/metrics_chart.html', config=config)

In [35]:
trace1 = go.Bar(
    x=['Не переглянуті до кінця', 'Переглянуті до кінця'],
    y=[1811982, 3004447],
    name='Distribution',
#     marker=dict(
#         color='black'
#     )
)


data = [trace1]
layout = go.Layout(
    title='Розподіл переглянутих фільмів та серіалів')


config={'showLink': False, 'displayModeBar': False}

fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='plots/distrib.html', config=config)

In [60]:
data_matrix = [[' ', 'Precision@10','Recall@10','MAP@10', 'NDCG@10'],
               ['The second level', 0.0425413, 0.113818, 0.0718531, 0.0945739],
               ['The first level', 0.038066, 0.106846, 0.0541142, 0.0861713],
               ['The most popular', 0.0130161, 0.0234716, 0.00832245, 0.0191037]]

colorscale = [[0, '#3779d1'],[.5, '#ffffff'],[1, '#ffffff']]

config={'showLink': False, 'displayModeBar': False}
table = ff.create_table(data_matrix, index=True,  colorscale=colorscale)
for i in range(len(table.layout.annotations)):
    table.layout.annotations[i].font.size = 18

iplot(table, filename='plots/metrics_table')
#plot(table, filename='plots/metrics_table.html', config=config, auto_open=False)

In [61]:
trace0 = go.Bar(
    x=['Precision@10','Recall@10','MAP@10', 'NDCG@10'],
    y=[0.0425413, 0.113818, 0.0718531, 0.0945739],
    name='The second level',
    marker=dict(
        color='#78a55a'
    )
)

trace1 = go.Bar(
    x=['Precision@10','Recall@10','MAP@10', 'NDCG@10'],
    y=[0.038066, 0.106846, 0.0541142, 0.0861713],
    name='The first level',
    marker=dict(
        color='#f29d38'
    )
)
trace2 = go.Bar(
    x=['Precision@10','Recall@10','MAP@10', 'NDCG@10'],
    y=[0.0130161, 0.0234716, 0.00832245, 0.0191037],
    name='The most popular',
    marker=dict(
        color='#3779d1'
    )
)



data = [trace2, trace1, trace0]
layout = go.Layout(
    barmode='group',
    title='Порівняння методів'
)
config={'showLink': False, 'displayModeBar': False}

fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='plots/metrics_chart.html', config=config)